In [1]:
testcode=[  1641,    393,   8629,   8491,  26086,   2296,    663,   1588,
        25764,   8533,   4981,   6187,   3268,   4237,    343,   3078,
         6643, 106002,    338,   3554]

In [2]:
import os
import pandas as pd
import math
import findspark

findspark.init('/usr/hdp/2.6.0.3-8/spark')

In [5]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import rdd, SparkContext, SparkFiles
sc=SparkContext()

In [7]:
def single_test0(userID, codes):
    results=[]
#     userID = np.random.randint(1000,2000)
    for i in codes:
        results.append((u"{},{},1".format(userID,i)))
    print len(results)
    return results

In [10]:
new = sc.parallelize(single_test0(2000, testcode))
type(new)

20


pyspark.rdd.RDD

In [13]:
new.take(3)

[u'2000,1641,1', u'2000,393,1', u'2000,8629,1']

In [15]:
fl = sc.textFile('file:///home/soyoung/HackerInsurTech/sample.csv')
print type(fl)
header = fl.first() #extract header
fl = fl.filter(lambda x: x!=header)

<class 'pyspark.rdd.RDD'>


In [16]:
fl.take(3)

[u'1,31,0', u'1,1029,0', u'1,1061,0']

In [21]:
data = sc.union([fl,new])
data.take(3)

[u'1,31,0', u'1,1029,0', u'1,1061,0']

___

In [22]:
ratings = data.map(lambda l: l.split(',')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [23]:
rank = 10
numIterations = 10


In [24]:
%%time
model = ALS.train(ratings, rank, numIterations)

CPU times: user 16 ms, sys: 8 ms, total: 24 ms
Wall time: 5.18 s


In [25]:
ratings.take(1)

[Rating(user=1, product=31, rating=0.0)]

In [26]:
ratings.take(1)[0][0:2]

(1, 31)

___

In [27]:
def single_test(userID, codes):
    results=[]
#     userID = np.random.randint(1000,2000)
    for i in codes:
        results.append((userID, i))
    print len(results)
    return results

In [28]:
ratings.filter(lambda x: x[0] == 2000).map(lambda p: (p[0], p[1]))

PythonRDD[232] at RDD at PythonRDD.scala:43

In [29]:
%%time
results = model.predictAll(ratings.filter(lambda x: x[0] == 2000).map(lambda p: (p[0], p[1])))

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 1.55 s


In [30]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType, IntegerType, FloatType

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

schema = StructType([StructField('userID', IntegerType(), True), 
                     StructField('codeID', IntegerType(), True), 
                     StructField('valid_prob', FloatType(), True)])



In [31]:
df = sqlContext.createDataFrame(results, schema)
df

DataFrame[userID: int, codeID: int, valid_prob: float]

___

In [32]:
%%time
codes = pd.read_csv('./sample.csv', usecols=['codeID'])['codeID'].unique()

CPU times: user 20 ms, sys: 12 ms, total: 32 ms
Wall time: 38.5 ms


In [33]:
sc.parallelize([(1, 2), (3, 4)]).collect()

[(1, 2), (3, 4)]

In [34]:
%%time
results = model.predictAll(sc.parallelize(single_test(2000, codes)))

9066
CPU times: user 124 ms, sys: 0 ns, total: 124 ms
Wall time: 255 ms


In [35]:
df = sqlContext.createDataFrame(results, schema)
dfp =df.toPandas()
dfp

,userID,codeID,valid_prob
0,2000,57972,0.264440
1,2000,1084,0.689283
2,2000,3456,0.547365
3,2000,6400,-0.153344
4,2000,3272,1.385956
5,2000,142192,0.786680
6,2000,81132,0.236345
7,2000,6308,0.365598
8,2000,44828,0.000000
9,2000,52328,0.456694


In [36]:
dfp[dfp['valid_prob']<0]

,userID,codeID,valid_prob
3,2000,6400,-0.153344
24,2000,26012,-0.050069
26,2000,5324,-0.263530
45,2000,6840,-0.034632
46,2000,4628,-0.154884
48,2000,6940,-0.758555
73,2000,2912,-0.654457
80,2000,101864,-0.833476
81,2000,196,-0.237067
86,2000,1176,-0.636076


In [37]:
dfp[dfp['valid_prob']<0.5]

,userID,codeID,valid_prob
0,2000,57972,0.264440
3,2000,6400,-0.153344
6,2000,81132,0.236345
7,2000,6308,0.365598
8,2000,44828,0.000000
9,2000,52328,0.456694
10,2000,69720,0.000000
11,2000,155820,0.000000
13,2000,1040,0.106191
20,2000,6644,0.000000


In [38]:
dfp[dfp['valid_prob']>0.95]

,userID,codeID,valid_prob
4,2000,3272,1.385956
17,2000,140,1.213815
69,2000,5508,1.192402
87,2000,3296,1.005494
92,2000,4304,1.549970
106,2000,1320,1.011420
110,2000,4424,0.985857
114,2000,54272,1.489392
119,2000,81564,1.012300
136,2000,32596,1.326495
